In [1]:
import os
import re 
import jieba
from bs4 import BeautifulSoup

In [22]:
def runUdnSegment(dataroot, outputfile, action, unwanted_ptn):
    '''Extract UDN news, and ouput char-level segement file 
    
    Need to revise pattern regular experission 
    
    Args:
        dataroot (str): the position of UDN news (recursive)
        outputfile (str): the position of output file 
        action (dict): parameter for seperate and segmentation-level 
        unwanted_ptn (rgex): pattern for kicking unwanted character 
    Return: 
        None
    '''
    
    sep_method = action.get('sep_method', 'comma')
    seg_level = action.get('seg_level', 'char')
    
    if os.path.exists(outputfile): 
        print('Clean %s' %(outputfile))
        os.remove(outputfile)
    
    pattern = re.compile('TEXT')
    
    for dirPath, dirName, filelist in os.walk(dataroot, topdown=False):
        if pattern.search(dirPath):        
            print(dirPath)
            for file in filelist:

                inputfile = dirPath+'/'+file
    #             print(inputfile)
                with open(inputfile, 'rb') as fp:
                    data = fp.read().decode('big5-hkscs', 'ignore')
                    soup = BeautifulSoup(data, 'lxml')
                    
                content = contentExtract(soup)
#                 seqs = seperateSeq(content, sep_method)
#                 seqs_filter = filterSeq(seqs, unwanted_ptn)
#                 seg_string = transformSeq(seqs_filter, seg_level)
                seg_string = transformSeq(content, seg_level)

                
                with open(outputfile, 'a', encoding='utf8') as wp:
                    wp.write(seg_string+'\n')


In [3]:
def contentExtract(soup):
    '''Extract the string content of website 
    Args:
        soup (Beatutifulsoup): website 
    Return:
        output (str): the string content of website
    '''
    output = ''
    for pTxt in soup.find_all('p'):
        res = ''
        for tag_c in pTxt.contents:
            try:
                if tag_c.get('class')==1:
                    res = res+tag_c.string
            except:
                res = res + tag_c
        res = res.strip('.\f\n\r\t\v')
        if len(res)==0:
            continue
        output = output + res+'\n'
#         print(output)
    return output

In [4]:
def seperateSeq(content, sep_method):
    '''Seperate string into sub-sentence
    Args:
        content (str): website content 
        sep_method (str): 'original' or 'comma'
    Return:
        output (list): sub-sentence
    '''
    output = []
    if sep_method == 'comma':
        pattern = re.compile('[，。！？]')
        content = ''.join(content.split('\n'))

        pre_idx=0
        for idx, ch in enumerate(content):
            if pattern.search(ch):
                tmp = content[pre_idx:idx+1]
                output.append(tmp)
                pre_idx = idx+1

    #     print(pre_idx, len(seq))
        if pre_idx<len(content):
            tmp = content[pre_idx:len(content)]
            output.append(tmp)
    elif sep_method:
        output = content.split('\n')
    
    return output 

In [5]:
def filterSeq(lst, pattern):
    '''Filter unwanted sequence based on pattern
    Args:
        lst (list): the list of website seperated content 
        pattern (rgex): the pattern we don't want 
    Return:
        output (list): list after filter
    '''
    
    output = [seq for seq in lst if not pattern.search(seq)]
    
    if not output:
        return list()
    
    if output[0].find('】'): 
        _ = output.pop(0)
    
    return output 

In [24]:
def transformSeq(seqs, seg_level):
    '''filter the line existed Unwatned pattern, and seperate the char with "space"
    Args:
        seqs (list): list from website 
        seg_level: which lm-level ('word' OR 'char)
    Return:
        output (str): string which have been seperated by 'space'
    '''
    output = list()
    if seg_level == 'word':
        for seq in seqs:
            segs = jieba.cut(seq)
            output.append(' '.join(segs))            
    elif seg_level == 'char':
        for seq in seqs:
            output.append(' '.join(seq))
    return ' '.join(output)

In [7]:
def runSinica(inputfile, outputfile):
    with open(inputfile, 'r', encoding='utf8') as fp,\
    open(outputfile, 'w', encoding='utf8') as wp:
        for line in fp:
            data = line.strip('\n').split(' ')
            output = []
            for item in data:
                tmp = item.split('|')
                if len(tmp)==2:
                    output.append(tmp[0])

            wp.write(' '.join(output))
            wp.write('\n')
    #         print(' '.join(output))
        

In [25]:
if __name__=='__main__':
    ptn = re.compile('[1]')
    par = {
        'sep_method':'comma'
        , 'seg_level':'char'}
    
#     data_root = sys.argv[1]
#     output_file = sys.argv[2]  
    data_root = '/home/kiwi/udn_data/Files/'
    output_file = './tmp.txt'
    
    runUdnSegment(dataroot=data_root, outputfile=output_file,
                 action=par, unwanted_ptn=ptn)
    

#     dataroot = '/home/kiwi/Documents/udn_data/Files/'
#     runCharSegment(dataroot,outputfile,pattern)
#     runWordSegment(dataroot,'./lm_data/seg_all.txt')
#     inputfile = '/home/kiwi/udn_data/training/sinica.corpus.txt'
#     runSinica(inputfile,'./lm_data/sinica_word.txt')

Clean ./tmp.txt
/home/kiwi/udn_data/Files/20160618/TEXT
記者許家瑜
暴雨澆不熄富福里居民對「街友勸離」專案的執著，離時間還有半個鐘頭，守望相助隊副隊長謝文興抵達康定路派出所看到記者不斷強調「為了守護里民的權益
「今天街友一定很少下雨天應該很快結束巡邏」簽到後穿上反光背心鴨舌帽口哨、拿手電筒或閃光指揮棒握拳高「街友離開、地方繁榮就來」出發時雨恰好也停了。
警民六人穿梭潮濕巷道中，街友是否路邊。出發不到五分鐘，洪傳雄發現穿橘色外套的街友坐在陰暗台階上，腳邊放著一個大塑膠袋「阿伯你都沒洗澡身上都臭烘烘」隊員拿著閃光棒蹲低靠近，酸臭汗味撲鼻而來。
「你一定要洗澡，可以去梧州街不要在這邊睡覺不要等我們走後再回來唷！」可以去中和收容中心，有吃睡你臭烘烘沒人要跟你睡一起」。
三角公園，看到一對年輕情侶坐有感而發說「若是被街友霸佔，公園就是死掉了」曾有外地遊客看到街友打赤膊躺在公園內，嚇得不敢靠近。走沒幾步路，大家木質座椅上紙箱飯盒「有些街友吃便當只吃雞腿、排骨，其他隨便亂丟造成髒亂。
大夥邊說邊走來到寺前公園，路邊變電箱圍籬內的街友「請將家當一併帶走」一百五十公分高的圍籬掩不住街友憤怒情緒，他隨手將行李、黑色後背包扔到另一頭，鐵條響起「噹噹」兩聲。
洪傳雄靠著圍籬問街友，「你真的沒地方住嗎？別人」沒說完「如果我們角色對換你會怎麼樣」。為什麼不去遊民收容中心？很舒適，「那是真正需要的人去住」。
隊員好奇街友為何不去找工作，他突然憤怒大吼「我也想找工作啊，你又怎麼知道我的苦衷在哪不要趕人趕到走投無路」眼看街友情緒越來越激動，勸離小組沒再多問，下個勸離點。
兩年邁街友倒臥門前熟睡，隊員手電筒光線落在他們身上，謝文興高音量說：「這裡有貼禁止睡覺標示，阿伯快起來！」其中一名老翁坐起身長嘆一口氣，抓著殘障坡道手把撐起身子，口中念念有詞「才剛剛躺下」，拄著雨傘緩步離去。
另名趙姓街友不理會，頭藏被子動也不動，但還是配合起身，邊收拾行李邊抗議「這邊睡了近三十年，從沒人趕過，這幾天突然要檢查要離開。」捲草蓆夾在腋下轉身尋覓另一處棲身之所。
守望相助隊勸離趙姓街友後，為期八天的勸離行動告一段落，大家

記 者 許 家 瑜 
 暴 雨 澆 不 熄 富 福 里 居 民 對 「 街 友 勸 離 」 專 案 的 執 著 ， 離 時 間 還 有 半 個 